In [22]:
from bertopic import BERTopic
import pickle as pkl
import pandas as pd
import numpy as np
import random
from bertopic.vectorizers import ClassTfidfTransformer

In [23]:
# get original reviews
with open("../reviews.pkl", "rb") as file:
    reviews = pkl.load(file)
    print(reviews[0])

This was the first place in Vegas where the Yelpers let me down. The salt and pepper shrimp appetizer was inedible: all of the shell was left on underneath the crispy fried coating. As a result, the sauce used to marinate the shrimp never penetrated the meat, it coagulated in the head, and EXPLODED on you upon taking a bite. GROSS! The potstickers and the beef chow fun lacked any flavor whatsoever. The only redeeming quality (the service is pretty awful) is the roast duck: crispy and tasty skin with relatively moist meat and a relative bargain compared to places in NY and L.A.


In [36]:
%run ../../NER/NER.py 

In [37]:
%%python ../../NER/NER.py
from NER import NER

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [38]:
import tqdm

processed_text = []
processed_text_all = []
ner = NER()
counter = 0
for review in tqdm.tqdm(reviews, leave=False):
    flag, output = ner.preprocess_text(review)
    processed_text_all.append(output)
    if flag:
        counter += 1
        processed_text.append(output)
    

print(counter)
print(len(reviews))

OSError: [E050] Can't find model 'en_core_web_trf'. It doesn't seem to be a Python package or a valid path to a data directory.

In [5]:
# get the cleaned reviews 
with open("clean_reviews.pkl", "rb") as file:
    clean_reviews = pkl.load(file)
    print(clean_reviews[0])

['dr', 'goldberg', 'offer', 'everything', 'look', 'for', 'in', 'general', 'practitioner', 'he', 'nice', 'and', 'easy', 'to', 'talk', 'to', 'without', 'being', 'patronizing', 'he', 'always', 'on', 'time', 'in', 'seeing', 'his', 'patient', 'he', 'affiliated', 'with', 'top', 'notch', 'hospital', 'nyu', 'which', 'my', 'parent', 'have', 'explained', 'to', 'me', 'is', 'very', 'important', 'in', 'case', 'something', 'happens', 'and', 'you', 'need', 'surgery', 'and', 'you', 'can', 'get', 'referral', 'to', 'see', 'specialist', 'without', 'having', 'to', 'see', 'him', 'first', 'really', 'what', 'more', 'do', 'you', 'need', 'sitting', 'here', 'trying', 'to', 'think', 'of', 'any', 'complaint', 'have', 'about', 'him', 'but', 'really', 'drawing', 'blank']


In [6]:
print(len(clean_reviews))
# index_list = range(len(clean_reviews))
# random.sample(index_list, 10000)
random.seed(10701)
clean_reviews_down = random.sample(clean_reviews, 10000)
print(len(clean_reviews_down))

700000
10000


In [59]:
# get data in string type
processed_corpus_1 = [" ".join(tokens) for tokens in clean_reviews_down]

In [40]:
# get the cleaned reviews 
with open("base.tag", "rb") as file:
    tagged_reviews = pkl.load(file)
    print(tagged_reviews[0])

['first', 'place', 'vega', '<person>', 'let', 'salt', 'pepper', 'shrimp', 'appetizer', 'inedible', 'shell', 'leave', 'underneath', 'crispy', 'fry', 'coating', 'result', 'sauce', 'use', 'marinate', 'shrimp', 'never', 'penetrate', 'meat', 'coagulate', 'head', 'explode', 'upon', 'take', 'bite', 'gross', 'potstickers', 'beef', 'chow', 'fun', 'lack', 'flavor', 'whatsoever', 'redeeming', 'quality', 'service', 'pretty', 'awful', 'roast', 'duck', 'crispy', 'tasty', 'skin', 'relatively', 'moist', 'meat', 'relative', 'bargain', 'compare', 'place', 'ny', 'la']


In [60]:
# get tagged reviews in string type
processed_corpus_2 = [" ".join(tokens) for tokens in tagged_reviews]

In [67]:
# unguided topic modeling

unguided_topic_model = BERTopic()
topics, probs = unguided_topic_model.fit_transform(processed_corpus_2)

In [68]:
unguided_topic_model.get_topic(0)

[('order', 0.009495104222183918),
 ('table', 0.008440899712858524),
 ('restaurant', 0.008034488095463569),
 ('food', 0.007607394272051359),
 ('ask', 0.007509774826434069),
 ('come', 0.0069323314578481985),
 ('menu', 0.006812663321105426),
 ('server', 0.006792388385898539),
 ('eat', 0.006438739360711522),
 ('one', 0.00637333992700273)]

In [70]:
unguided_topic_model.get_topic_info().to_csv('unguided_topics.csv', index=False)

In [79]:
# method 1: guided topic modeling

seed_topic_list = [
    ["pizza", "food", "chicken", "burger", "taco", "salsa", "mexican", "chip", "bean", "burrito", "enchilada", "rice", "tortilla", "guacamole", "carne", "asada", "shrimp", "lobster", "sushi", "roll", "fish", "sashimi", "tuna", "tempura"],
    ["doctor", "office", "medical", "service", "patient", "health", "insurance", "client", "car", "vehicle"], 
    ["hair", "nail", "salon", "gel", "polish", "manicure"], #maybe counts as service? 
    "de le la u00e9 et est u00e0 un com pa www que da en die yelp u00e9e du select une pour service au mais plus".split(" "), #french
    "bar drink beer night bartender table music atmosphere".split(" "), #bars
    "hotel bed bath shower spa pool casino strip night desk check club show sexy men girl".split(" "),#vegas
    "store shop price item shopping buy need sale mall product boutique outlet walmart".split(" "),#shopping malls
]


topic_model = BERTopic(seed_topic_list=seed_topic_list, embedding_model="all-MiniLM-L6-v2")
topics, probs = topic_model.fit_transform(processed_corpus_2)

In [80]:
topic_model.get_topic(0)

[('store', 0.02396121600159621),
 ('car', 0.019250964345364538),
 ('org', 0.01401766080860879),
 ('shop', 0.013073270195769284),
 ('call', 0.01262408014445486),
 ('buy', 0.01222551386951358),
 ('customer', 0.012205159147548847),
 ('need', 0.011820163282104043),
 ('person', 0.01164278172858635),
 ('service', 0.011484148602410716)]

In [81]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3991,-1_food_org_place_good,"[food, org, place, good, go, get, like, order,...",[locate inside <fac> downtown la vega <org> pl...
1,0,1249,0_store_car_org_shop,"[store, car, org, shop, call, buy, customer, n...",[bad place go nothing particular staff managem...
2,1,1109,1_food_order_table_come,"[food, order, table, come, get, restaurant, se...",[put one line <org> 3star restaurant posing 5s...
3,2,707,2_room_hotel_stay_casino,"[room, hotel, stay, casino, bed, pool, desk, o...",[boyfriend first time vega without family nnwe...
4,3,370,3_mexican_salsa_taco_burrito,"[mexican, salsa, taco, burrito, bean, chip, fo...",[wow im begin think great mexican food az move...
...,...,...,...,...,...
65,64,12,64_ramen_noodle_broth_rice,"[ramen, noodle, broth, rice, pork, shio, chash...",[like clean modern feel <org> huge communal ta...
66,65,12,65_dance_club_vip_security,"[dance, club, vip, security, drink, dj, kick, ...",[<org> hat fun midst identity crisis specifica...
67,66,11,66_clean_smell_teriyaki_table,"[clean, smell, teriyaki, table, remodel, cloth...",[food okay service horrible table clean seat g...
68,67,10,67_sandwich_meat_turkey_sub,"[sandwich, meat, turkey, sub, italian, bread, ...",[<org> little atmosphere customer service grea...


In [94]:
topic_model.get_topic_info().to_csv('tag_review_guided.csv', index=False)

In [83]:
topic_model.visualize_topics()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'store | car | org | shop | call', 1249],
                                   [1, 'food | order | table | come | get', 1109],
                                   [2, 'room | hotel | stay | casino | bed', 707],
                                   ...,
                                   [66, 'clean | smell | teriyaki | table | remodel', 11],
                                   [67, 'sandwich | meat | turkey | sub | italian', 10],
                                   [68, 'food | sit | would | quickly | bar', 10]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': array([1249, 1109,  707,  370,  188,  160,  154,  153,  121,  120,  100,   99,
                                          95,   90,   81,   79,   79,   73,   70,   69,   68,   60,   56,   56,
                                          54,   54,   50,   48,   48,   36,   35,   34,   34,   34,   33,   32,
                                          29,   29,   28,   27,   27,   24,   21,   21,   21,   19,   18,   18,
                                          17,   17,   17,   17,   16,   16,   16,   16,   15,   15,   14,   14,
                                          13,   13,   12,   12,   12,   12,   11,   10,   10]),
                         'sizemode': 'area',
                         'sizeref': 0.780625,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([  9.328935  ,   0.02937774,   9.863922  ,  -0.30276433,  -0.81183946,
                            0.10614217,  -5.665723  ,  -0.034917  ,  12.130159  ,  10.925153  ,
                          -12.587723  ,  11.448222  ,   4.592939  ,   4.108864  ,  -1.9489045 ,
                            0.75315905,   9.780348  ,  12.111827  ,  16.426895  ,   5.137704  ,
                            9.128333  ,  -0.88303745,  -1.6592183 ,  16.49822   ,  13.921603  ,
                            0.6889582 ,  -0.03187177,   4.1666994 ,  -0.0316983 ,  -0.14814435,
                            0.17519023,  11.003358  ,   5.195626  ,   0.6889603 ,   9.5197935 ,
                            9.409526  ,   9.492021  ,  11.486329  ,   4.2396145 ,  -0.11520915,
                           -5.6698713 ,   0.16830629,   4.033137  ,   9.19728   ,   9.786066  ,
                           14.140067  ,  16.619633  ,  -0.11885917,   0.4778838 ,  11.114753  ,
                            0.3300852 ,  -2.0122736 ,   5.29615   ,   4.5532565 ,   0.89078903,
                            4.7076535 ,  14.080869  ,  10.8133745 ,  10.939945  ,  14.222405  ,
                            4.7416215 ,  -1.573075  ,   4.168387  , -12.587817  ,  -2.1049678 ,
                           11.009405  ,   4.632     ,  -0.21405832,   4.617067  ], dtype=float32),
              'xaxis': 'x',
              'y': array([-1.825547  , 17.601948  , -1.552932  , 17.211136  , 16.929504  ,
                          16.924374  , 11.297594  , 16.448355  ,  6.512755  , -1.2901732 ,
                          -7.8798623 , 12.807101  , -1.4250034 ,  4.050965  ,  3.6077902 ,
                          16.802135  , -1.3971881 ,  6.513367  ,  5.492082  ,  7.5813904 ,
                          -1.7024102 , 16.917843  ,  3.3748538 ,  5.5624914 , -7.37709   ,
                          -8.960271  , 17.156494  ,  4.084235  , 17.332472  , 16.34947   ,
                          17.68128   , -0.87067133,  7.5253873 , -8.960278  ,  1.8681546 ,
                           1.9772062 ,  1.8956455 , 12.769451  ,  4.084301  ,  9.459464  ,
                          11.293293  , 17.043003  ,  4.295509  , -1.9280269 , -1.6006713 ,
                

In [95]:
# saving the top reviews

# Get full document info
review_info = topic_model.get_document_info(processed_corpus_2)

# For a specific topic (e.g., topic 0)
topic_documents = review_info[review_info.Topic == -1]
topic_documents = pd.DataFrame(topic_documents)

topic_documents['Document'].to_csv('uncluster_review_guided.csv', index=False, header=False)

unclustered_review = topic_documents['Document']

In [96]:
# performing unguided clustering on the unclustered_review
topics, probs = unguided_topic_model.fit_transform(unclustered_review)


In [98]:
unguided_topic_model.get_topic_info().to_csv('unguided_unclustered_clusters.csv', index=False)

In [46]:
# method 2: seeded words 
# https://maartengr.github.io/BERTopic/getting_started/seed_words/seed_words.html

# we can also specify the number of topics using a customized UMAP model
# umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

ctfidf_model = ClassTfidfTransformer(
    seed_words=["pizza", "doctor", "salon", "casino", "car"], 
    seed_multiplier=2
)

# We run the topic model with the seeded words
seed_topic_model = BERTopic(
    min_topic_size=10,
    ctfidf_model=ctfidf_model,
).fit(processed_corpus_2)


In [50]:
seed_topic_model.get_topic(3)

[('mexican', 0.03583264237461713),
 ('salsa', 0.03151384238804967),
 ('taco', 0.029816057285831477),
 ('burrito', 0.025546936126877286),
 ('chip', 0.018284862435536182),
 ('bean', 0.017794452142008216),
 ('food', 0.014786150846995727),
 ('tortilla', 0.014093107649477557),
 ('margarita', 0.012944764061113483),
 ('good', 0.012157636122235497)]

In [53]:
seed_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4467,-1_org_go_get_good,"[org, go, get, good, place, food, like, time, ...",[second last time visit <org> food always dece...
1,0,647,0_room_hotel_casino_stay,"[room, hotel, casino, stay, bed, org, desk, po...",[get back stay <org> last night never stay ive...
2,1,596,1_order_table_food_restaurant,"[order, table, food, restaurant, come, good, m...",[place okay menu seem little generic lack anyt...
3,2,511,2_bar_club_drink_beer,"[bar, club, drink, beer, dance, music, night, ...",[place favorite mine since start go bar defini...
4,3,339,3_mexican_salsa_taco_burrito,"[mexican, salsa, taco, burrito, chip, bean, fo...",[wow im begin think great mexican food az move...
...,...,...,...,...,...
72,71,11,71_shake_wife_steak_taste,"[shake, wife, steak, taste, pudding, naan, lam...",[boyfriend ate <org> anniversary dinner young ...
73,72,11,72_happy_hour_wing_slider,"[happy, hour, wing, slider, yeah, poki, spicy,...",[happy hour actual joke rule fair happy hour p...
74,73,11,73_fly_flight_airline_free,"[fly, flight, airline, free, attendant, portla...",[ive always love <org> since little girl fly f...
75,74,11,74_close_business_sign_830pm,"[close, business, sign, 830pm, reopen, 2011, d...",[want try since reopen sadly disappointed walk...


In [58]:
seed_topic_model.get_topic_info().to_csv('seedtopics.csv', index=False)

In [54]:
seed_topic_model.visualize_topics()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'room | hotel | casino | stay | bed', 647],
                                   [1, 'order | table | food | restaurant | come', 596],
                                   [2, 'bar | club | drink | beer | dance', 511],
                                   ...,
                                   [73, 'fly | flight | airline | free | attendant', 11],
                                   [74, 'close | business | sign | 830pm | reopen', 11],
                                   [75, 'crab | leg | seafood | shrimp | steam', 11]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': array([647, 596, 511, 339, 218, 212, 197, 183, 178, 168, 157, 135, 134, 117,
                                        109, 109,  95,  85,  79,  76,  71,  69,  68,  64,  60,  58,  57,  57,
                                         50,  49,  49,  49,  42,  41,  37,  36,  35,  35,  33,  32,  30,  29,
                                         28,  27,  25,  24,  24,  23,  23,  23,  22,  21,  21,  21,  19,  19,
                                         17,  17,  17,  17,  17,  16,  16,  16,  15,  15,  15,  15,  12,  12,
                                         11,  11,  11,  11,  11,  11]),
                         'sizemode': 'area',
                         'sizeref': 0.404375,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([ 3.4176402e+00,  4.8758874e+00,  3.7400718e+00, -6.7809262e+00,
                           1.3561292e+01, -6.7908311e+00,  4.6663647e+00,  4.0573759e+00,
                          -6.8826499e+00, -6.8545275e+00, -7.8359184e+00,  1.4366772e+01,
                          -7.3758821e+00,  1.7629181e+01, -6.1225452e+00, -5.4202571e+00,
                           1.9768761e+01,  1.1353531e+01,  1.4411450e+01, -8.0250273e+00,
                          -7.3944764e+00,  1.4707418e+01, -5.7699089e+00, -5.8907123e+00,
                           4.3060689e+00, -7.3069282e+00, -6.8455472e+00,  1.4594931e+01,
                           4.0061159e+00,  4.8550181e+00,  1.1567361e-01,  1.4458728e+01,
                           3.4608510e+00, -8.7778997e+00,  4.3969169e+00,  4.9789529e+00,
                           9.1660900e+00,  1.3021637e-01,  9.2848581e-01, -8.8986788e+00,
                           1.3534143e+01, -8.7011271e+00,  1.7628473e+01,  1.3158245e-02,
                           3.1072197e+00, -6.1088815e+00, -6.6536846e+00,  1.3213231e+01,
                           1.9727713e+01,  3.7800364e+00, -7.3742938e+00,  5.4147482e+00,
                          -5.4199495e+00, -7.9621892e+00, -5.7308545e+00,  4.4817910e+00,
                           1.9867420e+01,  3.8284495e+00,  1.3333627e+01, -5.8666496e+00,
                           9.2818940e-01,  5.3424988e+00,  5.0345368e+00,  1.4663369e+01,
                           4.6152277e+00,  3.1200848e+00,  2.6166272e-01,  9.1660938e+00,
                           4.9490113e+00,  1.1353570e+01, -8.6173792e+00, -7.4781361e+00,
                           3.8809087e+00,  1.4441803e+01,  4.4149442e+00, -6.6389089e+00],
                         dtype=float32),
              'xaxis': 'x',
              'y': array([ 8.887036 ,  6.005782 ,  8.460435 ,  7.8117495, -2.851496 ,  7.906014 ,
                           4.725848 ,  7.8302016,  8.404039 ,  7.571895 , -3.663094 ,  6.6701674,
                           6.900309 ,  6.9766417,  7.712135 , -7.8544583,  7.2461386, 18.599285 ,
                           6.7161303, -3.8524141,  7.9515524,  7.0146556,  7.62292  ,  8.660879 ,
                           7.5511036,  7.1592383,  8.472247 ,  